# DSLab Homework 1 - Data Science with CO2

## Hand-in Instructions

- __Due: 23.03.2021 23h59 CET__
- `git push` your final verion to the master branch of your group's Renku repository before the due
- check if `Dockerfile`, `environment.yml` and `requirements.txt` are properly written
- add necessary comments and discussion to make your codes readable

## Carbosense

The project Carbosense establishes a uniquely dense CO2 sensor network across Switzerland to provide near-real time information on man-made emissions and CO2 uptake by the biosphere. The main goal of the project is to improve the understanding of the small-scale CO2 fluxes in Switzerland and concurrently to contribute to a better top-down quantification of the Swiss CO2 emissions. The Carbosense network has a spatial focus on the City of Zurich where more than 50 sensors are deployed. Network operations started in July 2017.

<img src="http://carbosense.wdfiles.com/local--files/main:project/CarboSense_MAP_20191113_LowRes.jpg" width="500">

<img src="http://carbosense.wdfiles.com/local--files/main:sensors/LP8_ZLMT_3.JPG" width="156">  <img src="http://carbosense.wdfiles.com/local--files/main:sensors/LP8_sensor_SMALL.jpg" width="300">

## Description of the homework

In this homework, we will curate a set of **CO2 measurements**, measured from cheap but inaccurate sensors, that have been deployed in the city of Zurich from the Carbosense project. The goal of the exercise is twofold: 

1. Learn how to deal with real world sensor timeseries data, and organize them efficiently using python dataframes.

2. Apply data science tools to model the measurements, and use the learned model to process them (e.g., detect drifts in the sensor measurements). 

The sensor network consists of 46 sites, located in different parts of the city. Each site contains three different sensors measuring (a) **CO2 concentration**, (b) **temperature**, and (c) **humidity**. Beside these measurements, we have the following additional information that can be used to process the measurements: 

1. The **altitude** at which the CO2 sensor is located, and the GPS coordinates (latitude, longitude).

2. A clustering of the city of Zurich in 17 different city **zones** and the zone in which the sensor belongs to. Some characteristic zones are industrial area, residential area, forest, glacier, lake, etc.

## Prior knowledge

The average value of the CO2 in a city is approximately 400 ppm. However, the exact measurement in each site depends on parameters such as the temperature, the humidity, the altitude, and the level of traffic around the site. For example, sensors positioned in high altitude (mountains, forests), are expected to have a much lower and uniform level of CO2 than sensors that are positioned in a business area with much higher traffic activity. Moreover, we know that there is a strong dependence of the CO2 measurements, on temperature and humidity.

Given this knowledge, you are asked to define an algorithm that curates the data, by detecting and removing potential drifts. **The algorithm should be based on the fact that sensors in similar conditions are expected to have similar measurements.** 

## To start with

The following csv files in the `../data/carbosense-raw/` folder will be needed: 

1. `CO2_sensor_measurements.csv`
    
   __Description__: It containts the CO2 measurements `CO2`, the name of the site `LocationName`, a unique sensor identifier `SensorUnit_ID`, and the time instance in which the measurement was taken `timestamp`.
    
2. `temperature_humidity.csv`

   __Description__: It contains the temperature and the humidity measurements for each sensor identifier, at each timestamp `Timestamp`. For each `SensorUnit_ID`, the temperature and the humidity can be found in the corresponding columns of the dataframe `{SensorUnit_ID}.temperature`, `{SensorUnit_ID}.humidity`.
    
3. `sensor_metadata.csv`

   __Description__: It contains the name of the site `LocationName`, the zone index `zone`, the altitude in meters `altitude`, the longitude `lon`, and the latitude `lat`. 

Import the following python packages:

In [168]:
import pandas as pd
import numpy as np
import sklearn
import plotly.express as px
import plotly.graph_objects as go
import os

In [169]:
pd.options.mode.chained_assignment = None

## PART I: Handling time series with pandas (10 points)

### a) **8/10**

Merge the `CO2_sensor_measurements.csv`, `temperature_humidity.csv`, and `sensors_metadata.csv`, into a single dataframe. 

* The merged dataframe contains:
    - index: the time instance `timestamp` of the measurements
    - columns: the location of the site `LocationName`, the sensor ID `SensorUnit_ID`, the CO2 measurement `CO2`, the `temperature`, the `humidity`, the `zone`, the `altitude`, the longitude `lon` and the latitude `lat`.

| timestamp | LocationName | SensorUnit_ID | CO2 | temperature | humidity | zone | altitude | lon | lat |
|:---------:|:------------:|:-------------:|:---:|:-----------:|:--------:|:----:|:--------:|:---:|:---:|
|    ...    |      ...     |      ...      | ... |     ...     |    ...   |  ... |    ...   | ... | ... |



* For each measurement (CO2, humidity, temperature), __take the average over an interval of 30 min__. 

* If there are missing measurements, __interpolate them linearly__ from measurements that are close by in time.

__Hints__: The following methods could be useful

1. ```python 
pandas.DataFrame.resample()
``` 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html
    
2. ```python
pandas.DataFrame.interpolate()
```
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html
    
3. ```python
pandas.DataFrame.mean()
```
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html
    
4. ```python
pandas.DataFrame.append()
```
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html

In [170]:
!git lfs pull

In [202]:
measurements = pd.read_csv("../data/carbosense-raw/CO2_sensor_measurements.csv",
                             sep="\t",
                             parse_dates=['timestamp'])
measurements = measurements.rename({'SensorUnit_ID':'sensor', 'timestamp':'Timestamp'}, axis=1)

In [203]:
measurements

,Timestamp,LocationName,sensor,CO2
0,2017-10-01 00:05:00,ZGHD,1014,593.321562
1,2017-10-01 00:15:00,ZGHD,1014,595.846258
2,2017-10-01 00:25:00,ZGHD,1014,595.845353
3,2017-10-01 00:45:00,ZGHD,1014,599.039129
4,2017-10-01 00:55:00,ZGHD,1014,594.996629
...,...,...,...,...
188961,2017-10-31 23:03:00,ZLDW,1299,476.821183
188962,2017-10-31 23:13:00,ZLDW,1299,477.659209
188963,2017-10-31 23:23:00,ZLDW,1299,479.250364
188964,2017-10-31 23:33:00,ZLDW,1299,471.565620


In [204]:
measurements = measurements.set_index('Timestamp').groupby('sensor') \
              .apply(lambda df: df.interpolate(axis=0).resample('30min').mean())

In [205]:
measurements

sensor         CO2
sensor Timestamp                              
1014   2017-10-01 00:00:00  1014.0  595.004391
       2017-10-01 00:30:00  1014.0  597.017879
       2017-10-01 01:00:00  1014.0  596.487010
       2017-10-01 01:30:00  1014.0  598.787639
       2017-10-01 02:00:00  1014.0  595.366394
...                            ...         ...
1299   2017-10-31 21:30:00  1299.0  477.307329
       2017-10-31 22:00:00  1299.0  474.396830
       2017-10-31 22:30:00  1299.0  474.406102
       2017-10-31 23:00:00  1299.0  477.910252
       2017-10-31 23:30:00  1299.0  471.881151

[66667 rows x 2 columns]

In [206]:
temperature = pd.read_csv("../data/carbosense-raw/temperature_humidity.csv",
                          sep="\t",
                          parse_dates=['Timestamp'])

In [207]:
metadata = pd.read_csv("../data/carbosense-raw/sensors_metadata.csv", sep="\t")
metadata.head()

,LocationName,zone,altitude,lat,lon
0,AJGR,2,627.8,47.33321,8.480300
1,BSCR,2,462.3,47.35581,8.430070
2,BUDF,10,490.2,47.36270,8.433110
3,KBRL,7,510.5,47.32582,8.537550
4,KTGM,2,530.3,47.32334,8.596422


In [208]:
melted = pd.melt(temperature, id_vars='Timestamp')
melted['variable'] = melted['variable'].apply(lambda s: s.split('.'))
melted[['sensor','var']] = pd.DataFrame(melted.variable.tolist(), index= melted.index)
melted = melted[['Timestamp','value','sensor','var']]
melted = melted.pivot(index=['Timestamp','sensor'], columns='var', values='value').reset_index()

In [209]:
melted.head()

var,Timestamp,sensor,humidity,temperature
0,2017-10-01,1014,92.262787,11.39
1,2017-10-01,1015,91.949982,10.55
2,2017-10-01,1016,92.613739,11.02
3,2017-10-01,1017,89.811844,11.05
4,2017-10-01,1018,91.427368,11.38


In [210]:
melted = melted.set_index('Timestamp').groupby('sensor') \
              .apply(lambda df: df.interpolate(axis=0).resample('30min').mean())

In [211]:
melted

var                          humidity  temperature
sensor Timestamp                                  
1014   2017-10-01 00:00:00  92.440806    11.396667
       2017-10-01 00:30:00  91.625097    11.303333
       2017-10-01 01:00:00  91.096761    11.240000
       2017-10-01 01:30:00  91.275734    11.260000
       2017-10-01 02:00:00  91.003937    11.285000
...                               ...          ...
1299   2017-10-31 21:30:00  78.965388     3.923333
       2017-10-31 22:00:00  79.702896     3.766667
       2017-10-31 22:30:00  80.769740     3.493333
       2017-10-31 23:00:00  81.676366     3.373333
       2017-10-31 23:30:00  81.910970     3.286667

[68448 rows x 2 columns]

In [220]:
merged = pd.merge(melted, measurements, how='inner', left_on=['sensor','Timestamp'], right_on=['sensor','Timestamp'])

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [218]:
melted = melted.reset_index()
measurements = measurements.drop('sensor',axis=1).reset_index()

In [219]:
measurements

,sensor,Timestamp,CO2
0,1014,2017-10-01 00:00:00,595.004391
1,1014,2017-10-01 00:30:00,597.017879
2,1014,2017-10-01 01:00:00,596.487010
3,1014,2017-10-01 01:30:00,598.787639
4,1014,2017-10-01 02:00:00,595.366394
...,...,...,...
66662,1299,2017-10-31 21:30:00,477.307329
66663,1299,2017-10-31 22:00:00,474.396830
66664,1299,2017-10-31 22:30:00,474.406102
66665,1299,2017-10-31 23:00:00,477.910252


In [180]:
(merged.SensorUnit_ID == 1031).any()

True

In [144]:
merged = merged.drop(['Timestamp','SensorUnit_ID'], axis=1)

In [145]:
merged = merged.interpolate(axis=0)
merged.isnull().any().any()

False

In [146]:
# merged = pd.merge(merged, metadata, left_on='LocationName', right_on='LocationName')

In [147]:
merged = merged.set_index(['timestamp'])

In [148]:
merged = merged.groupby('sensor').resample('30min').mean()

In [159]:
merged.index = merged.index.droplevel(0)

True

### b) **2/10** 

Export the curated and ready to use timeseries to a csv file, and properly push the merged csv to Git LFS.

In [46]:
!cat -a ../

.     Dockerfile       .git	       .gitlab-ci.yml  .renku
..    .dockerignore    .gitattributes  notebooks       .renkulfsignore
data  environment.yml  .gitignore      README.md       requirements.txt


In [ ]:
pd.to_csv(os.path.join(DATAmerged, sep='\t')

## PART II: Data visualization (15 points)

### a) **5/15** 
Group the sites based on their altitude, by performing K-means clustering. 
- Find the optimal number of clusters using the [Elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)). 
- Wite out the formula of metric you use for Elbow curve. 
- Perform clustering with the optimal number of clusters and add an additional column `altitude_cluster` to the dataframe of the previous question indicating the altitude cluster index. 
- Report your findings.

__Note__: [Yellowbrick](http://www.scikit-yb.org/) is a very nice Machine Learning Visualization extension to scikit-learn, which might be useful to you. 

### b) **4/15** 

Use `plotly` (or other similar graphing libraries) to create an interactive plot of the monthly median CO2 measurement for each site with respect to the altitude. 

Add proper title and necessary hover information to each point, and give the same color to stations that belong to the same altitude cluster.

### c) **6/15**

Use `plotly` (or other similar graphing libraries) to plot an interactive time-varying density heatmap of the mean daily CO2 concentration for all the stations. Add proper title and necessary hover information.

__Hints:__ Check following pages for more instructions:
- [Animations](https://plotly.com/python/animations/)
- [Density Heatmaps](https://plotly.com/python/mapbox-density-heatmaps/)

## PART III: Model fitting for data curation (35 points)

### a) **2/35**

The domain experts in charge of these sensors report that one of the CO2 sensors `ZSBN` is exhibiting a drift on Oct. 24. Verify the drift by visualizing the CO2 concentration of the drifting sensor and compare it with some other sensors from the network. 

### b) **8/35**

The domain experts ask you if you could reconstruct the CO2 concentration of the drifting sensor had the drift not happened. You decide to:
- Fit a linear regression model to the CO2 measurements of the site, by considering as features the covariates not affected by the malfunction (such as temperature and humidity)
- Create an interactive plot with `plotly` (or other similar graphing libraries):
    - the actual CO2 measurements
    - the values obtained by the prediction of the linear model for the entire month of October
    - the __confidence interval__ obtained from cross validation
- What do you observe? Report your findings.

__Note:__ Cross validation on time series is different from that on other kinds of datasets. The following diagram illustrates the series of training sets (in orange) and validation sets (in blue). For more on time series cross validation, there are a lot of interesting articles available online. scikit-learn provides a nice method [`sklearn.model_selection.TimeSeriesSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html).

![ts_cv](https://player.slideplayer.com/86/14062041/slides/slide_28.jpg)

### c) **10/35**

In your next attempt to solve the problem, you decide to exploit the fact that the CO2 concentrations, as measured by the sensors __experiencing similar conditions__, are expected to be similar.

- Find the sensors sharing similar conditions with `ZSBN`. Explain your definition of "similar condition".
- Fit a linear regression model to the CO2 measurements of the site, by considering as features:
    - the information of provided by similar sensors
    - the covariates associated with the faulty sensors that were not affected by the malfunction (such as temperature and humidity).
- Create an interactive plot with `plotly` (or other similar graphing libraries):
    - the actual CO2 measurements
    - the values obtained by the prediction of the linear model for the entire month of October
    - the __confidence interval__ obtained from cross validation
- What do you observe? Report your findings.

### d) **10/35**

Now, instead of feeding the model with all features, you want to do something smarter by using linear regression with fewer features.

- Start with the same sensors and features as in question c)
- Leverage at least two different feature selection methods
- Create similar interactive plot as in question c)
- Describe the methods you choose and report your findings

### e) **5/35**

Eventually, you'd like to try something new - __Bayesian Structural Time Series Modelling__ - to reconstruct counterfactual values, that is, what the CO2 measurements of the faulty sensor should have been, had the malfunction not happened on October 24. You will use:
- the information of provided by similar sensors - the ones you identified in question c)
- the covariates associated with the faulty sensors that were not affected by the malfunction (such as temperature and humidity).

To answer this question, you can choose between a Python port of the CausalImpact package (such as https://github.com/dafiti/causalimpact) or the original R version (https://google.github.io/CausalImpact/CausalImpact.html) that you can run in your notebook via an R kernel (https://github.com/IRkernel/IRkernel).

Before you start, watch first the [presentation](https://www.youtube.com/watch?v=GTgZfCltMm8) given by Kay Brodersen (one of the creators of the causal impact implementation in R), and this introductory [ipython notebook](http://nbviewer.jupyter.org/github/dafiti/causalimpact/blob/master/examples/getting_started.ipynb) with examples of how to use the python package.

- Report your findings:
    - Is the counterfactual reconstruction of CO2 measurements significantly different from the observed measurements?
    - Can you try to explain the results?

# That's all, folks!